In [1]:
!nvidia-smi

Mon May  5 07:35:46 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   51C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

**Training the Model using VGG 19 Classification**

In [3]:
import tensorflow as tf
from tensorflow.keras.applications import VGG19
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
import matplotlib.pyplot as plt
import numpy as np
import os

# === Constants ===
IMG_SIZE = (224, 224)
BATCH_SIZE = 32  # Number of images processed together in one step
NUM_CLASSES = 6  # 5 soil types + NotSoil

# === Create the Model ===
def create_model():
    base_model = VGG19(weights='imagenet', include_top=False, input_shape=(*IMG_SIZE, 3))
    for layer in base_model.layers:
        layer.trainable = False

 # These are Custom classification Layers of the model
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(1024, activation='relu')(x)
    x = Dropout(0.5)(x)
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.3)(x)
    output = Dense(NUM_CLASSES, activation='softmax')(x)

    model = Model(inputs=base_model.input, outputs=output)

    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
        loss='categorical_crossentropy',
        metrics=['accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall()]
    )

    return model

# === Create Generators ===
def create_data_generators(train_dir, test_dir):
    train_datagen = ImageDataGenerator(
        rescale=1./255,
        rotation_range=20,
        width_shift_range=0.2,
        height_shift_range=0.2,
        horizontal_flip=True,
        vertical_flip=True,
        brightness_range=[0.8, 1.2],
        zoom_range=0.2,
        fill_mode='nearest'
    )

    test_datagen = ImageDataGenerator(rescale=1./255)

    train_gen = train_datagen.flow_from_directory(
        train_dir,
        target_size=IMG_SIZE,
        batch_size=BATCH_SIZE,
        class_mode='categorical',
        shuffle=True
    )

    test_gen = test_datagen.flow_from_directory(
        test_dir,
        target_size=IMG_SIZE,
        batch_size=BATCH_SIZE,
        class_mode='categorical',
        shuffle=False
    )

    return train_gen, test_gen

# === Plot Training History ===
def plot_training_history(history):
    plt.figure(figsize=(12, 4))

    # Accuracy
    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'], label='Train Accuracy')
    plt.plot(history.history['val_accuracy'], label='Val Accuracy')
    plt.title('Model Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()

    # Loss
    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'], label='Train Loss')
    plt.plot(history.history['val_loss'], label='Val Loss')
    plt.title('Model Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()

    plt.tight_layout()
    plt.savefig('training_history.png')
    plt.close()

# === Train Function ===
def train_model():
    train_dir = '/content/drive/MyDrive/SoilClassification_CropRecommendation/Dataset/Train'
    test_dir = '/content/drive/MyDrive/SoilClassification_CropRecommendation/Dataset/Test'

    train_gen, test_gen = create_data_generators(train_dir, test_dir)
    model = create_model()

    callbacks = [
        tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True),
        tf.keras.callbacks.ModelCheckpoint('best_model_checkpoint.h5', monitor='val_accuracy', save_best_only=True),
        tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=1e-6)
    ]

    history = model.fit(
        train_gen,
        epochs=70,
        validation_data=test_gen,
        callbacks=callbacks
    )

    # Save artifacts
    model.save('soil_classification_Updated_model_vgg19.h5')
    np.save('class_indices.npy', train_gen.class_indices)
    plot_training_history(history)

    return model, history, train_gen.class_indices

# === Main Execution ===
if __name__ == "__main__":
    model, history, class_indices = train_model()

    print("\nClass Mapping:")
    for class_name, index in class_indices.items():
        print(f"{index}: {class_name}")


Found 1488 images belonging to 6 classes.
Found 504 images belonging to 6 classes.
80134624/80134624 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/70
47/47 ━━━━━━━━━━━━━━━━━━━━ 0s 13s/step - accuracy: 0.2605 - loss: 1.7876 - precision: 0.3544 - recall: 0.0354 

47/47 ━━━━━━━━━━━━━━━━━━━━ 846s 18s/step - accuracy: 0.2617 - loss: 1.7861 - precision: 0.3553 - recall: 0.0355 - val_accuracy: 0.1250 - val_loss: 1.7750 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - learning_rate: 1.0000e-04
Epoch 2/70
47/47 ━━━━━━━━━━━━━━━━━━━━ 0s 773ms/step - accuracy: 0.3865 - loss: 1.5759 - precision: 0.5313 - recall: 0.0655

47/47 ━━━━━━━━━━━━━━━━━━━━ 41s 876ms/step - accuracy: 0.3863 - loss: 1.5761 - precision: 0.5299 - recall: 0.0652 - val_accuracy: 0.1944 - val_loss: 1.6237 - val_precision: 0.8125 - val_recall: 0.0258 - learning_rate: 1.0000e-04
Epoch 3/70
47/47 ━━━━━━━━━━━━━━━━━━━━ 0s 741ms/step - accuracy: 0.3645 - loss: 1.5705 - precision: 0.5290 - recall: 0.0662

...

47/47 ━━━━━━━━━━━━━━━━━━━━ 39s 833ms/step - accuracy: 0.7601 - loss: 0.6434 - precision: 0.8364 - recall: 0.6872 - val_accuracy: 0.7778 - val_loss: 0.6671 - val_precision: 0.8307 - val_recall: 0.7202 - learning_rate: 1.0000e-04
Epoch 33/70
47/47 ━━━━━━━━━━━━━━━━━━━━ 39s 837ms/step - accuracy: 0.8064 - loss: 0.6231 - precision: 0.8731 - recall: 0.7238 - val_accuracy: 0.7778 - val_loss: 0.6823 - val_precision: 0.8213 - val_recall: 0.7202 - learning_rate: 1.0000e-04
Epoch 34/70
47/47 ━━━━━━━━━━━━━━━━━━━━ 39s 837ms/step - accuracy: 0.7545 - loss: 0.6971 - precision: 0.8479 - recall: 0.6724 - val_accuracy: 0.7560 - val_loss: 0.7427 - val_precision: 0.7885 - val_recall: 0.6806 - learning_rate: 1.0000e-04
Epoch 35/70
47/47 ━━━━━━━━━━━━━━━━━━━━ 0s 740ms/step - accuracy: 0.7748 - loss: 0.6570 - precision: 0.8384 - recall: 0.6937

47/47 ━━━━━━━━━━━━━━━━━━━━ 40s 841ms/step - accuracy: 0.7749 - loss: 0.6566 - precision: 0.8387 - recall: 0.6937 - val_accuracy: 0.7877 - val_loss: 0.6783 - val_precision: 0.8299 - val_recall: 0.7163 - learning_rate: 1.0000e-04
Epoch 36/70
47/47 ━━━━━━━━━━━━━━━━━━━━ 40s 845ms/step - accuracy: 0.8063 - loss: 0.6047 - precision: 0.8749 - recall: 0.6988 - val_accuracy: 0.7857 - val_loss: 0.6651 - val_precision: 0.8192 - val_recall: 0.7282 - learning_rate: 2.0000e-05
Epoch 37/70
47/47 ━━━━━━━━━━━━━━━━━━━━ 39s 838ms/step - accuracy: 0.7839 - loss: 0.6649 - precision: 0.8482 - recall: 0.7085 - val_accuracy: 0.7877 - val_loss: 0.6554 - val_precision: 0.8146 - val_recall: 0.7321 - learning_rate: 2.0000e-05
Epoch 38/70
47/47 ━━━━━━━━━━━━━━━━━━━━ 39s 834ms/step - accuracy: 0.7788 - loss: 0.6194 - precision: 0.8616 - recall: 0.7077 - val_accuracy: 0.7877 - val_loss: 0.6541 - val_precision: 0.8263 - val_recall: 0.7361 - learning_rate: 2.0000e-05
Epoch 39/70
47/47 ━━━━━━━━━━━━━━━━━━━━ 0s 709ms/step

47/47 ━━━━━━━━━━━━━━━━━━━━ 39s 827ms/step - accuracy: 0.7903 - loss: 0.6194 - precision: 0.8554 - recall: 0.6956 - val_accuracy: 0.7917 - val_loss: 0.6494 - val_precision: 0.8230 - val_recall: 0.7381 - learning_rate: 2.0000e-05
Epoch 40/70
47/47 ━━━━━━━━━━━━━━━━━━━━ 39s 817ms/step - accuracy: 0.7935 - loss: 0.6037 - precision: 0.8798 - recall: 0.7204 - val_accuracy: 0.7897 - val_loss: 0.6524 - val_precision: 0.8184 - val_recall: 0.7421 - learning_rate: 2.0000e-05
Epoch 41/70
47/47 ━━━━━━━━━━━━━━━━━━━━ 0s 712ms/step - accuracy: 0.7911 - loss: 0.6039 - precision: 0.8605 - recall: 0.6936

47/47 ━━━━━━━━━━━━━━━━━━━━ 39s 831ms/step - accuracy: 0.7911 - loss: 0.6041 - precision: 0.8605 - recall: 0.6938 - val_accuracy: 0.7937 - val_loss: 0.6436 - val_precision: 0.8216 - val_recall: 0.7401 - learning_rate: 2.0000e-05
Epoch 42/70
47/47 ━━━━━━━━━━━━━━━━━━━━ 40s 827ms/step - accuracy: 0.8039 - loss: 0.5930 - precision: 0.8807 - recall: 0.7262 - val_accuracy: 0.7917 - val_loss: 0.6367 - val_precision: 0.8208 - val_recall: 0.7361 - learning_rate: 2.0000e-05
Epoch 43/70
47/47 ━━━━━━━━━━━━━━━━━━━━ 38s 810ms/step - accuracy: 0.8018 - loss: 0.6215 - precision: 0.8747 - recall: 0.7158 - val_accuracy: 0.7897 - val_loss: 0.6435 - val_precision: 0.8184 - val_recall: 0.7421 - learning_rate: 2.0000e-05
Epoch 44/70
47/47 ━━━━━━━━━━━━━━━━━━━━ 40s 846ms/step - accuracy: 0.8002 - loss: 0.6314 - precision: 0.8668 - recall: 0.7181 - val_accuracy: 0.7877 - val_loss: 0.6450 - val_precision: 0.8216 - val_recall: 0.7401 - learning_rate: 2.0000e-05
Epoch 45/70
47/47 ━━━━━━━━━━━━━━━━━━━━ 39s 824ms/ste


Class Mapping:
0: Alluvial soil
1: Black Soil
2: Clay soil
3: NotSoil
4: Red soil
5: Yellow Soil


**The Below code is for Testing the Model**

In [4]:
#Testing code
import tensorflow as tf
import numpy as np
import cv2
from tensorflow.keras.models import load_model

def load_and_preprocess_image(image_path):
    """Load and preprocess an image for prediction"""
    img = cv2.imread(image_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (224, 224))
    img = img / 255.0
    return np.expand_dims(img, axis=0)

def predict_image(model, image_path, class_indices):
    """Predict the class of an image"""
    # Load and preprocess image
    img = load_and_preprocess_image(image_path)

    # Get prediction
    predictions = model.predict(img)

    # Get class index and confidence
    class_idx = np.argmax(predictions[0])
    confidence = predictions[0][class_idx]

    # Get class name
    class_names = {v: k for k, v in class_indices.items()}
    predicted_class = class_names[class_idx]

    return predicted_class, confidence

def main():
    # Load model and class indices
    model = load_model('soil_classification_Updated_model_vgg19.h5')
    class_indices = np.load('class_indices.npy', allow_pickle=True).item()

    # Test images
    test_images = [
        '/content/drive/MyDrive/SoilClassification_CropRecommendation/Dataset/Train/Alluvial soil/Alluvial_1.jpg',
        '/content/drive/MyDrive/SoilClassification_CropRecommendation/Dataset/Train/NotSoil/Turmeric 3.jpg',
        '/content/drive/MyDrive/SoilClassification_CropRecommendation/Dataset/Train/NotSoil/istockphoto-1090614334-612x612.jpg'
    ]

    # Perform predictions
    for image_path in test_images:
        predicted_class, confidence = predict_image(model, image_path, class_indices)
        print(f"\nImage: {image_path}")
        print(f"Predicted class: {predicted_class}")
        print(f"Confidence: {confidence*100:.2f}%")

        if predicted_class == 'non_soil' or confidence < 0.7:
            print("This is not a soil image.")

if __name__ == "__main__":
    main()

1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step

Image: /content/drive/MyDrive/SoilClassification_CropRecommendation/Dataset/Train/Alluvial soil/Alluvial_1.jpg
Predicted class: Alluvial soil
Confidence: 48.40%
This is not a soil image.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step

Image: /content/drive/MyDrive/SoilClassification_CropRecommendation/Dataset/Train/NotSoil/Turmeric 3.jpg
Predicted class: NotSoil
Confidence: 99.75%
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step

Image: /content/drive/MyDrive/SoilClassification_CropRecommendation/Dataset/Train/NotSoil/istockphoto-1090614334-612x612.jpg
Predicted class: NotSoil
Confidence: 72.91%


In [5]:
import numpy as np
class_indices = np.load('class_indices.npy', allow_pickle=True).item()
print(class_indices)


{'Alluvial soil': 0, 'Black Soil': 1, 'Clay soil': 2, 'NotSoil': 3, 'Red soil': 4, 'Yellow Soil': 5}
